In [2]:
import pandas as pd
import numpy as np
import math

In [30]:
data = pd.read_csv("AB_test_data.csv")
data['purchase_TF'] = list(map(int, list(data['purchase_TF'])))

# Question 1

In [31]:
convert_tb = pd.crosstab(data.purchase_TF,data.Variant, margins=True)
convert_tb

Variant,A,B,All
purchase_TF,,,
0,106298,4117,110415
1,18702,883,19585
All,125000,5000,130000


In [32]:
sizeA = list(convert_tb.A)[2]
sizeB = list(convert_tb.B)[2]
size = list(convert_tb.All)[2]
convertA = list(convert_tb.A)[1]
convertB = list(convert_tb.B)[1]
convert = list(convert_tb.All)[1]
conversion_all = convert/size
conversion_A = convertA/sizeA
conversion_B = convertB/sizeB

In [33]:
z_test = (conversion_B - conversion_A)/np.sqrt(conversion_A*(1-conversion_A)/sizeB)

In [34]:
z_test

5.349273094732516

In [35]:
###Accept H1, alternative B improved conversion rates

# Question 2

In [36]:
###Set minimum detectable effect = conversion_B-conversion_A(0.026984)

In [37]:
mde = conversion_B-conversion_A
t_alpha_over_2 = 1.96 # alpha/2 = 0.05/2 = 0.025 
t_beta = 0.842 # beta = -0.2. 
pbar = (conversion_A+conversion_B)/2
min_n = pow(t_alpha_over_2*np.sqrt(2*pbar*(1-pbar))+t_beta*np.sqrt(conversion_A*(1-conversion_A)+conversion_B*(1-conversion_B)),2)/pow(mde,2)

In [38]:
size_sample = math.ceil(min_n)
size_sample
###The optimal sample size is 2943 when alpha = 0.05, beta = 0.2, minimum detectable effect = 0.0269

2943

In [39]:
data_onlyB = data[data['Variant'] == 'B']

In [40]:
data_onlyB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 1 to 129960
Data columns (total 4 columns):
purchase_TF    5000 non-null int64
Variant        5000 non-null object
date           5000 non-null object
id             5000 non-null object
dtypes: int64(1), object(3)
memory usage: 195.3+ KB


In [41]:
z_test_list = []
sample_list = []
for i in range(1,11):   
    data_sample = data_onlyB.sample(n=math.ceil(size_sample),axis = 0)
    sample_list.append(data_sample)
    conversion_sample = sum(data_sample.purchase_TF)/size_sample # converted / all samples
    z_test = (conversion_sample - conversion_A)/np.sqrt(conversion_A*(1-conversion_A)/size_sample)
    z_test_list.append(z_test)  
z_test_list

[4.066055276557851,
 2.619062575993062,
 4.376125140964587,
 3.9626986550889347,
 4.324446830230131,
 3.497593858478827,
 4.272768519495675,
 3.859342033620023,
 4.272768519495675,
 4.376125140964587]

In [42]:
###Accept H1 every time

# Question 3

In [59]:
from math import log
A = log(1/0.05) # upper bound A = 1/alpha, where alpha = 0.05
B = log(0.2) # lower bound B = beta = 1-power = 0.2

In [60]:
###Assume xi are Bernoulli with P(xi =1) = p0 (0.149616) under H0 and P(xi =1) = p0+mde (0.19961) under H1.

In [61]:
results = [] # H0 or H1
iterations = []
for sample in sample_list:
    lam = 0 
    n = 1
    for x in list(sample.purchase_TF):
        if x == 1:
            lambda_x = log((conversion_B)/conversion_A)
        else:
            lambda_x = log((1-conversion_B)/(1-conversion_A))
        lam = lam + lambda_x     
        if lam <= B:
            results.append('H0')
            iterations.append(n)
            break
        elif lam >= A:
            results.append('H1')
            iterations.append(n)
            break
        elif n == size_sample and lam > B and lam < A:
            results.append('No result')
            iterations.append(n)
        n = n+1      

In [62]:
results

['H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1', 'H1']

In [63]:
iterations

[882, 269, 538, 799, 766, 1123, 493, 644, 1755, 1385]

In [64]:
sum(iterations)/len(iterations)

865.4

In [71]:
###Stop the test prior to using the full sample for every sample.
###Average number of iterations: 681.8